In [1]:
import csv
import gzip
from IPython.display import clear_output

In [2]:
import csv

ids = dict()
with open('pfam_domains_grouped.csv') as f:
    reader = csv.reader(f)
    next(reader)
    for id, _ in reader:
        ids[id] = None

In [3]:
def read_fasta(file, ids):
    # file: a path to a fasta file
    # ids a dict of fasta_ids you are interested in of the form { 'fasta_id': None, ... }
    fasta_name = None
    seq = None
    
    with open('interpro_seqs.fasta', 'a') as fOut:
        with gzip.open(file, "rt") as fhIn:
            for idx, line in enumerate(fhIn):
                if idx % 1000000 == 0:
                    clear_output()
                    print(f'Processing Line: {idx}')
                if line.startswith(">"):
                    if fasta_name:
                        # do something with the previous seq
                        if fasta_name in ids:
                            # calc length here
                            seq_len = len(seq)
                            ids[fasta_name] = seq_len

                            fOut.write(f'>{fasta_name}\n{seq}\n')

                    # reset the previous seq
                    fasta_name = line.split('|')[1]
                    seq = ''
                else:
                    seq = seq+line.rstrip()

        # calc the length for the final seq here
        seq_len = len(seq)
        ids[fasta_name] = seq_len
        fOut.write(f'>{fasta_name}\n{seq}\n')
        return(ids)

In [4]:
id_out = read_fasta('uniprot_trembl.fasta.gz', ids)

Processing Line: 1809000000


In [5]:
id_out2 = read_fasta('uniprot_sprot.fasta.gz', id_out)

Processing Line: 4000000


In [6]:
with open('pfam_domains_grouped.csv') as f:
    with open('pfam_domains_grouped_len.csv', 'w') as f_out:
        reader = csv.reader(f)
        next(reader)
        for idx, line in enumerate(reader):
            f_out.write(','.join(line)+','+str(id_out2[line[0]])+'\n')
            if idx % 1000000 == 0:
                clear_output()
                print(f'Processing Line: {idx}')

Processing Line: 182000000
